# Python & Heuristics
June 6, 2022  
`matthias@hasler.fr`

Activities:

- Testing
- More on `list` and `dict`
- Simple bank model
- Efficiency & sorting
- Change problem

# Testing

We will use 2 third-party libraries: `pytest` and `hypothesis`.

From you terminal window, run `python -m pip install pytest hypothesis`

## [pytest](https://docs.pytest.org/en/7.1.x/)

pytest is the most popular Python testing library.  
It looks for functions named `test_xxx` inside files named `test_yyy.py`, and uses the `assert` keyword to mark tests. Example:

In [ ]:
# file: test_fib.py
from myfibonacci import fib

def test_fib10():
    assert fib(10) == 55, "my error msg"

In your terminal, run `pytest test_fib.py`.  
If you need more information, `pytest -svv test_fib.py` will:

- `[s]how` your print statements and
- be `[v]ery [v]erbose` about the assertion that failed.

## [hypothesis](https://hypothesis.readthedocs.io/en/latest/quickstart.html)

hypothesis is a test generator suite.  
I heard about it before, but tried it yesterday for the first time.  
It runs tests with random parameters and tries to find the simplest failing examples.  
Productivity boost for algorithms and data structures.

# More containers

Examples: `simpsons.py` and `containers2.py`

Exercise: `more_containers.py`

Test your code with `pytest test_more_containers.py`

# Bank model

Complete the class definition: `bank.py`

Test your code with `pytest test_bank.py`

# Efficiency & sorting

- Fibonacci with memoization
- Bubble sort
- Merge sort
- Bonus: Quicksort
- Bonus++: Heapsort

## Memoization
check out `fibo.py`

## Bubble sort

- For every `i`, swap `(arr[i], arr[i+1])` if `arr[i] > arr[i+1]`
- If there was at least one swap, restart at beginning of line.

![](https://upload.wikimedia.org/wikipedia/commons/5/54/Sorting_bubblesort_anim.gif)

## Merge sort

- Divide array in two halfs
- Sort each half recursively
- Combine sorted arrays greedily, by inserting the smaller of the 2 heads

![](https://upload.wikimedia.org/wikipedia/commons/c/cc/Merge-sort-example-300px.gif?20151222172210)

## optional: Quicksort

- Take the first element `pivot`.
- Split array into `x <= pivot` and `x > pivot`
- Sort each part and combine, pivot in the middle
- What's the worst case?

![](https://upload.wikimedia.org/wikipedia/commons/6/6a/Sorting_quicksort_anim.gif)

## very optional: Heapsort

We can use indices to implicitely store a binary tree structure: `k` is the parent of `2*k+1` and `2*k+2`
- **heapify** a list: make sure that `arr[k] >= max(arr[2*k+1], arr[2*k+2])` for all k (hint: start from the end)
- implement **heap_pop**:
    - swap first and last element
    - pop last element (old head)
    - swap new head with its largest child if it is greater than head, and recurse
- to sort a list, first heapify, then pop_head till empty.

![](https://upload.wikimedia.org/wikipedia/commons/1/1b/Sorting_heapsort_anim.gif)

# Change problem

The problem has 2 parts:

- Given `exact_change` and `(a,b,c,d)` the denominations, find the smallest number of coins `na+nb+nc+nd` required to come up with `exact_change == a*na + b*nb + c*nc + d*nd`
- Over all possible denominations `(a,b,c,d)`, find the one that minimizes the expected number of coins required to get the exact change on average, knowing that multiples of 5 are N times more likely.

We can optimize part 1 using **dynamic programming**

## part 1: greedy is not optimal

`pytest test_change.py`

In [1]:
# Find minimum number of coins required

def optimal_naive(exact_change, a,b,c,d):
    assert a == 1
    best = (exact_change, 0, 0, 0)
    
    for nd in range(exact_change//d + 1):
        rem1 = exact_change - nd * d
        for nc in range(rem1//c + 1):
            rem2 = rem1 - nc * c
            nb = rem2 // b
            na = rem2 % b
            alt = (na, nb, nc, nd)
            if sum(alt) < sum(best):
                best = alt
    return sum(best)

## part 2: minimize over (a,b,c,d)

In [2]:
# Score one configuration: expected number of coins required

def avg_coins(max_change, N, a,b,c,d):
    num = 0
    den = 0
    for change in range(max_change):
        weight = N if change % 5 == 0 else 1
        num += weight * optimal_naive(change, a,b,c,d)
        den += weight
    return num / den

In [3]:
# Find best configuration

# TODO: pip install tqdm
from tqdm import tqdm  # progress bar

max_change = 100
N = 2.5

a = 1
best = (1e9, (1,1,1,1))
for b in tqdm(range(2, max_change)):
    for c in range(b+1, max_change):
        for d in range(c+1, max_change):
            score = avg_coins(max_change, N, a,b,c,d)
            alt = (score, (a,b,c,d))
            if alt < best:
                best = alt
best

100%|███████████████████████████████████████████| 98/98 [00:23<00:00,  4.12it/s]


(3.6769230769230767, (1, 5, 19, 30))

## Optimization: Dynamic programming

Idea:

- suppose `(na,nb,nc,nd)` is optimal for `X = a*na + b*nb + c*nc + d*nd`
- `{X-a, X-b, X-c, X-d}` all require at least `na+nb+nc+nd-1` coins, otherwise there is a better solution for `X`
- one of `{X-a, X-b, X-c, X-d}` requires `na+nb+nc+nd-1` coins, otherwise there is no solution for `X`

We can construct optimal solutions for larger numbers from smaller optimal solutions, by adding one coin.

In [1]:
from ipywidgets import interact

def show_step(stop):
    # at the beginning, it is only possible to make 0 change
    cost = [99]*20
    cost[0] = 0
    
    a, b, c, d = 1, 2, 4, 7
    
    # we base the solution for i on previous solutions
    for i in range(stop):
        for j in [i-a, i-b, i-c, i-d]:
            if j >= 0:
                cost[i] = min(cost[i], cost[j]+1)
    
    header = [' '] * 20
    header[i] = 'v' # str(cost[i])
    for j in (i-a, i-b, i-c, i-d):
        if j >= 0:
            header[j] = str(cost[j])
    print(' '+'  '.join(header))
    print(cost[:i+1])
interact(show_step, stop=(1,20));

interactive(children=(IntSlider(value=10, description='stop', max=20, min=1), Output()), _dom_classes=('widget…